### The purpose of this notebook to is to recreate the regress the SSH in GLORYS onto the CWI developed in Amaya et al. (2022)

In [1]:
import os, gc; from os.path import exists
os.chdir('/vortexfs1/home/anthony.meza/CTWPC/scripts')
main_dir = "/vortexfs1/home/anthony.meza/CTWPC"
plotsdir = lambda x="": main_dir + "/plots/" + x
GLORYS_dir = lambda x="": main_dir + "/GLORYS_data" + x
GLORYS_data_dir = lambda x="": main_dir + "/GLORYS_processed/" + x
ERA5_data_dir = lambda x="": main_dir + "/ERA5_data/" + x

In [2]:
# from help_funcs import * 
# from eofs.xarray import Eof
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import importlib
import xarray as xr
import seaborn as sns
import pandas as pd
import cmocean.cm as cm
import netCDF4 as nc
import matplotlib.pyplot as plt
import gc
import os 
import multiprocessing
from os.path import exists
sns.set_context("notebook")
import dask_labextension
from pathlib import Path
import numpy as np
import dask
# dask.config.set({"array.slicing.split_large_chunks": False})

In [3]:
def remove_climatology(ds, climatology):
    ds = ds.convert_calendar('noleap') #remove leap years from operations
    anomalies = ds.groupby("time.dayofyear") - climatology
    anomalies["time"] = anomalies.indexes['time'].to_datetimeindex()
    anomalies["time"] = anomalies.indexes['time'].normalize()
    return anomalies

In [4]:
from dask_jobqueue import SLURMCluster  # setup dask cluster 
cluster = SLURMCluster(
    cores=36,
    processes=1,
    memory='100GB',
    walltime='02:00:00',
    queue='compute',
    interface='ib0')
print(cluster.job_script())
cluster.scale(jobs=8)
from dask.distributed import Client
client = Client(cluster)

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -n 1
#SBATCH --cpus-per-task=36
#SBATCH --mem=94G
#SBATCH -t 02:00:00

/vortexfs1/home/anthony.meza/mambaforge/envs/atm_rivers/bin/python -m distributed.cli.dask_worker tcp://172.16.3.125:36380 --nthreads 36 --memory-limit 93.13GiB --name dummy-name --nanny --death-timeout 60 --interface ib0



In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.16.3.125:36380,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


### We read in the Amaya Coastal Wave Index Amplitude $(\text{Index} =  \sqrt{\text{PC1}^2 + \text{PC2}^2)}$ and Phase $(\text{Phase} = atan( {\frac{\text{PC2}}{\text{PC1}}}))$ and recompute $\text{PC1}$ and $\text{PC2}$. We also remove the leap-year days from the GLORYS dataset.

In [16]:
import pandas as pd
rmm = pd.read_csv("rMII_index_latest.txt", delim_whitespace=True)
new_rmm = rmm.iloc[:, 3:]
new_rmm["time"] = pd.to_datetime(rmm.iloc[:, 0:3].astype("string").apply("-".join, axis=1))
rmm = new_rmm.rename(columns={new_rmm.columns[0]: "PC1", 
                       new_rmm.columns[1]: "PC2",
                       new_rmm.columns[2]: "Phase"})
rmm["Amplitude"] = np.sqrt(rmm["PC1"]**2 + rmm["PC2"]**2)
rmm = rmm.set_index("time")
rmm = rmm.to_xarray().isel(time = np.where(~np.isnan(rmm.Amplitude))[0])
rmm = rmm.sel(time = slice("1993", "2018"))

In [17]:
rmm = rmm.convert_calendar('noleap')
rmm["time"] = rmm.indexes['time'].to_datetimeindex()
rmm["time"] = rmm.indexes['time'].normalize()

/tmp/ipykernel_248947/2850264681.py:2: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  rmm["time"] = rmm.indexes['time'].to_datetimeindex()


In [18]:
def is_djf(month):
    return (month >= 1) & (month <= 2) | (month == 12)
def is_djfm(month):
    return (month >= 1) & (month <= 3) | (month == 12)
def is_jfm(month):
    return (month >= 1) & (month <= 3) 

In [19]:
rmm_Idx_djf = rmm.isel(time = is_djfm(rmm["time.month"])); min_amp = 1.5

phases_dict = {}
phases_dict["1"] = np.argwhere((rmm_Idx_djf.Phase.values == 1) & (rmm_Idx_djf.Amplitude.values > min_amp))
phases_dict["2"] = np.argwhere((rmm_Idx_djf.Phase.values == 2) & (rmm_Idx_djf.Amplitude.values > min_amp))
phases_dict["3"] = np.argwhere((rmm_Idx_djf.Phase.values == 3) & (rmm_Idx_djf.Amplitude.values > min_amp))
phases_dict["4"] = np.argwhere((rmm_Idx_djf.Phase.values == 4) & (rmm_Idx_djf.Amplitude.values > min_amp))
phases_dict["5"] = np.argwhere((rmm_Idx_djf.Phase.values == 5) & (rmm_Idx_djf.Amplitude.values > min_amp))
phases_dict["6"] = np.argwhere((rmm_Idx_djf.Phase.values == 6) & (rmm_Idx_djf.Amplitude.values > min_amp))
phases_dict["7"] = np.argwhere((rmm_Idx_djf.Phase.values == 7) & (rmm_Idx_djf.Amplitude.values > min_amp))
phases_dict["8"] = np.argwhere((rmm_Idx_djf.Phase.values == 8) & (rmm_Idx_djf.Amplitude.values > min_amp))

### Reading in the GLORYS and ERA5 data files. Files have been preprocessed and combined in order to take advantage of the **dask** feature of xarray. We also remove their seasonal cliamtologies

In [20]:
%%time 
def _preprocess(ds):
    return ds[["thetao"]].sel(latitude = slice(-2, 60), longitude = slice(-150, -75), time = slice("1993", "2018"))

glorys_anomalies = xr.open_mfdataset(
        GLORYS_data_dir("GLORYS_SST_AnomaliesBandPass.nc"),
        data_vars="minimal",
        coords="minimal",
        compat="override",
        preprocess=_preprocess,
        parallel=True,
        chunks={"time":320, "latitude":-1, "longitude":-1, "depth":1},
        engine="netcdf4")
glorys_anomalies = glorys_anomalies.sel(time = rmm.time.values)

CPU times: user 10.2 ms, sys: 1.8 ms, total: 12 ms
Wall time: 64.5 ms


In [21]:
def _preprocess_ERA5(ds):
    return ds.sel(latitude = slice(60, -2)).sel(longitude = slice(-150, -75)).sel(time = slice("1993", "2018"))

era5_anomalies = xr.open_mfdataset(GLORYS_data_dir("ERA5_AnomaliesBandPass.nc"), 
        data_vars="minimal", coords="minimal",
        compat="override", preprocess=_preprocess_ERA5,
        parallel=True, chunks={"longitude": -1, "latitude":-1, "time":320}, engine="netcdf4")

era5_anomalies = era5_anomalies.sel(time = rmm.time.values)

In [22]:
def _preprocess_ERA5(ds):
    return ds.sel(latitude = slice(60, -2)).sel(longitude = slice(-150, -75)).sel(time = slice("1993", "2018"))

era5_IVT_anomalies = xr.open_mfdataset(GLORYS_data_dir("ERA5_WaterAnomaliesBandPass.nc"), 
        data_vars="minimal", coords="minimal",
        compat="override", preprocess=_preprocess_ERA5,
        parallel=True, chunks={"longitude": -1, "latitude":-1, "time":320}, engine="netcdf4")

era5_IVT_anomalies = era5_IVT_anomalies.sel(time = rmm.time.values)
era5_IVT_anomalies = era5_IVT_anomalies.rename({"p71.162":"IVTE", "p72.162":"IVTN"})

In [23]:
DJF_phases = {}
var_keys = ["SST", "Precip", "TCWV", "z500", "IVTE", "IVTN"]
for key in var_keys:
    DJF_phases[key] = {}

In [24]:
for phase, phase_indices in phases_dict.items():
    
    num_phases_days = len(phase_indices)
    tp_anom_mean = era5_anomalies.tp.isel(time = is_djfm(rmm["time.month"])).isel(time = phase_indices.flatten())
    sst_anom_mean = glorys_anomalies.thetao.isel(time = is_djfm(rmm["time.month"])).isel(time = phase_indices.flatten())
    tcwv_anom_mean = era5_anomalies.tcwv.isel(time = is_djfm(rmm["time.month"])).isel(time = phase_indices.flatten())
    z500_anom_mean = era5_anomalies.z.isel(time = is_djfm(rmm["time.month"])).isel(time = phase_indices.flatten())
    IVTE_anom_mean = era5_IVT_anomalies.IVTE.isel(time = is_djfm(rmm["time.month"])).isel(time = phase_indices.flatten())
    IVTN_anom_mean = era5_IVT_anomalies.IVTN.isel(time = is_djfm(rmm["time.month"])).isel(time = phase_indices.flatten())

    DJF_phases["SST"][phase] = sst_anom_mean
    DJF_phases["Precip"][phase] = tp_anom_mean
    DJF_phases["TCWV"][phase] = tcwv_anom_mean
    DJF_phases["z500"][phase] = z500_anom_mean / 9.80
    DJF_phases["IVTE"][phase] = IVTE_anom_mean
    DJF_phases["IVTN"][phase] = IVTN_anom_mean
    print(phase, len(phase_indices))

1 115
2 144
3 185
4 154
5 150
6 141
7 213
8 181


In [25]:
for phase, phase_indices in phases_dict.items():
    print(phase)
    DJF_phases["SST"][phase] = DJF_phases["SST"][phase].compute()
    DJF_phases["Precip"][phase] = DJF_phases["Precip"][phase].compute()
    DJF_phases["TCWV"][phase] = DJF_phases["TCWV"][phase].compute()
    DJF_phases["z500"][phase] = DJF_phases["z500"][phase].compute()
    DJF_phases["IVTE"][phase] = DJF_phases["IVTE"][phase].compute()
    DJF_phases["IVTN"][phase] = DJF_phases["IVTN"][phase].compute()

1
2
3
4
5
6
7
8


## Getting the $\pm$ 3 days view for phase 5

In [26]:
# SST_DFJ_phase4_dict = {}
# precip_DFJ_phase4_dict = {}
# tcwv_DFJ_phase4_dict = {}
# z500_DFJ_phase4_dict = {}
# IVTE_DFJ_phase4_dict = {}
# IVTN_DFJ_phase4_dict = {}

# day_labels = np.array([-14, -7, 0, 7, 14])
# day_range = range(0, len(day_labels))
# renameindex = lambda x: x.reset_index(['time']).rename({'time': 'day'}).assign_coords(day = day_labels)

# SST_DFJ_phase4_dict["4"] = xr.zeros_like(renameindex(glorys_anomalies.thetao.isel(time = day_range)))
# precip_DFJ_phase4_dict["4"] = xr.zeros_like(renameindex(era5_anomalies.tp.isel(time = day_range)))
# tcwv_DFJ_phase4_dict["4"] = xr.zeros_like(renameindex(era5_anomalies.tcwv.isel(time = day_range)))
# z500_DFJ_phase4_dict["4"] = xr.zeros_like(renameindex(era5_anomalies.z.isel(time = day_range)))
# IVTE_DFJ_phase4_dict["4"] = xr.zeros_like(renameindex(era5_IVT_anomalies.IVTE.isel(time = day_range)))
# IVTN_DFJ_phase4_dict["4"] = xr.zeros_like(renameindex(era5_IVT_anomalies.IVTN.isel(time = day_range)))

In [17]:
# phase = "4"
# phase_indices = phases_dict[phase]
# num_phases_days = len(phase_indices)
# for idx in phase_indices:
#     dayindices = day_labels + idx[0]
#     maxtimen = len(era5_anomalies.tp.isel(time = is_djfm(rmm["time.month"])))
#     if (np.max(dayindices) < maxtimen) & (np.min(dayindices) > 0):
#         tp_anom_mean = renameindex(era5_anomalies.tp.isel(time = is_djfm(rmm["time.month"])).isel(time = dayindices))
#         sst_anom_mean = renameindex(glorys_anomalies.thetao.isel(time = is_djfm(rmm["time.month"])).isel(time = dayindices))
#         tcwv_anom_mean = renameindex(era5_anomalies.tcwv.isel(time = is_djfm(rmm["time.month"])).isel(time = dayindices))
#         z500_anom_mean = renameindex(era5_anomalies.isel(time = is_djfm(rmm["time.month"])).z.isel(time = dayindices))
#         IVTE_anom_mean = renameindex(era5_IVT_anomalies.IVTE.isel(time = is_djfm(rmm["time.month"])).isel(time = dayindices))
#         IVTN_anom_mean = renameindex(era5_IVT_anomalies.IVTN.isel(time = is_djfm(rmm["time.month"])).isel(time = dayindices))

#         SST_DFJ_phase4_dict[phase] += sst_anom_mean / num_phases_days
#         precip_DFJ_phase4_dict[phase] += tp_anom_mean / num_phases_days
#         tcwv_DFJ_phase4_dict[phase] += tcwv_anom_mean/ num_phases_days
#         IVTE_DFJ_phase4_dict[phase] += IVTE_anom_mean/ num_phases_days
#         IVTN_DFJ_phase4_dict[phase] += IVTN_anom_mean/ num_phases_days
#         z500_DFJ_phase4_dict[phase] += (z500_anom_mean / 1000) / num_phases_days
# print(phase, len(phase_indices))

4 154


In [27]:
# #constructing the full dictionary
# phase = "4"
# DJF_phase4_dict = {}
# var_keys = ["SST", "Precip", "TCWV", "z500", "IVTE", "IVTN"]
# for key in var_keys:
#     DJF_phase4_dict[key] = {}
# DJF_phase4_dict["SST"] = SST_DFJ_phase4_dict[phase].compute()
# DJF_phase4_dict["Precip"] = precip_DFJ_phase4_dict[phase].compute()
# DJF_phase4_dict["TCWV"] = tcwv_DFJ_phase4_dict[phase].compute()
# DJF_phase4_dict["z500"] = z500_DFJ_phase4_dict[phase].compute() 
# DJF_phase4_dict["IVTE"] = IVTE_DFJ_phase4_dict[phase].compute() 
# DJF_phase4_dict["IVTN"] = IVTN_DFJ_phase4_dict[phase].compute() 

In [28]:
# Store data (serialize)
INDEX_TYPE = "RMM"
indexdir = INDEX_TYPE + "_Phases"
try:
    os.mkdir(GLORYS_data_dir(indexdir))
except:
    print("dir already exists")
    
import pickle

with open(GLORYS_data_dir(INDEX_TYPE + "_Phases/Composites.pickle"), 'wb') as handle:
    pickle.dump(DJF_phases, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Store data (serialize)
# with open(GLORYS_data_dir(INDEX_TYPE + "_Phases/CompositesPhase4.pickle"), 'wb') as handle:
#     pickle.dump(DJF_phase4_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Store data (serialize)
with open(GLORYS_data_dir(INDEX_TYPE + "_Phases/IndexPhases.pickle"), 'wb') as handle:
    pickle.dump(phases_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

dir already exists
